In [9]:
P = Primes()
ZR.<x> = PolynomialRing(ZZ)
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def logsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

def generatingZeroLogsOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:
                        if mod(i,prime)>0:
                            data=data+[i]
    data=data.sort()
    return data



def generatingZeroLogsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:
                        if mod(i,prime)>0:
                            data=data+[[i,exponent]]
    return data




def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data



def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def padListOnRightToLengthK(listo,k):
    length=len(listo)
    padded=listo
   
    if length<k:
        for j in [length..k-1]:
            padded=padded+[0]
           
        return padded
    return listo

def testCandidateZeroLogOverField(f,prime,power,candidate):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    a=t^candidate
    expr3=sage_eval(expr,locals={'x':a})
    if expr3 == 0:tf=True
    return tf

def digitsToInteger(digitlist,prime):
    lenth=len(digitlist)
    sum=0
    for k in [0..lenth-1]:
        sum=sum+prime^k*digitlist[k]
    return sum

def minimumIntegerRepresentedInList(lstlst,prime):
    answer=Infinity
    ln=len(lstlst)
    for k in [0..ln-1]:
        lst=lstlst[k]
        n=digitsToInteger(lst,prime)
        if n < answer:answer=n
    return answer

def lastElement(lst):
    ln=len(lst)
    return lst[ln-1]

def push(lst):
    if len(lst)<2:return lst
    answer=[lastElement(lst)]
    for k in [0..len(lst)-2]:
        answer=flatten([answer]+[lst[k]])
    return answer

def cyclicPermutationEquivalenceClass(lst):
    if lst==[]:return lst
    answer=[lst]
    lnth=len(lst)
    for k in [0..lnth-2]:
        ple=push(lastElement(answer))
        answer=answer+[ple]
    return answer

def bestRepresentativeOfCyclicEquivalenceClass(lst,prime):
    cpec=cyclicPermutationEquivalenceClass(lst)
    minm=minimumIntegerRepresentedInList(cpec,prime)
    lcpec=len(lst) # ... = len(cpec)
    for k in [0..lcpec-1]:
        ck=cpec[k]
        dti=digitsToInteger(ck,prime)
        if minm==dti:return ck
    return "error"

def directZeroLogOverField(f,prime,power):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    answer=[]
    expr3=sage_eval(expr,locals={'x':0})
    if expr3 == 0:
        answer=[0]
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+[i]
    return answer

def reduce(f,prime):
    poly=polynomialnumerator(D(f))
    
    ls=list(poly)
    
    def modulo(n):
        return ZZ(mod(n,prime))
    def listmod(list):
        return map(modulo,list)
    lmls = listmod(ls)
    sm=0
    for j in [0..len(lmls)-1]:
        sm=sm+lmls[j]*x^j
    return sm


#McKay-Thompson series w. a(0) = 24.
mt=[
[-1, 1],
[0, 24],
[1, 276],
[2, 2048],
[3, 11202],
[4, 49152],
[5, 184024],
[6, 614400],
[7, 1881471],
[8, 5373952],
[9, 14478180],
[10, 37122048],
[11, 91231550],
[12, 216072192],
[13, 495248952],
[14, 1102430208],
[15, 2390434947],
[16, 5061476352],
[17, 10487167336],
[18, 21301241856],
[19, 42481784514],
[20, 83300614144],
[21, 160791890304],
[22, 305854488576],
[23, 573872089212],
[24, 1063005978624],
[25,1945403602764],
[26, 3519965179904],
[27, 6300794030460],
[28, 11164248047616],
[29, 19591528119288],
[30, 34065932304384],
[31, 58718797964805],
[32, 100372723007488],
[33, 170215559855424],
[34, 286470013685760],
[35, 478625723149576],
[36, 794110053826560],
[37, 1308745319975256],
[38, 2143055278039040],
[39, 3487563372381816],
[40, 5641848336678912],
[41, 9074553043554568],
[42, 14515166263443456],
[43, 23093778743102262],
[44, 36552977852071936],
[45, 57567784186189368],
[46, 90226777113575424],
[47, 140752796480416011],
[48, 218578429975461888],
[49, 337945040343588276],
[50, 520271697765971968],
[51, 797652526220573580],
[52, 1218002527825723392],
[53, 1852604006634050072],
[54, 2807138079496716288],
[55, 4237760460302936433],
[56, 6374456847628238848],
[57, 9554873766107770560],
[58, 14273181657059143680],
[59, 21250450411204068910],
[60, 31535729115847852032],
[61, 46650835290143061624],
[62, 68797209365301886976],
[63, 101150679669913197462],
[64, 148280443106626633728],
[65, 216743142763626253712],
[66, 315923191441199824896],
[67, 459218611940943755226],
[68, 665710603285072019456],
[69, 962508846974918603904],
[70, 1388038765923851599872],
[71, 1996639069403279491427],
[72, 2864978197116521938944],
[73, 4100990608911708903432],
[74, 5856297079648098807808],
[75, 8343432715970391209502],
[76, 11859696700297921757184],
[77, 16820105145987654631552],
[78, 23802835313046730063872],
[79, 33611779636250175278886],
[80, 47362494062244172660736],
[81, 66600077798590855556532],
[82, 93460562353103053049856],
[83, 130891485964083426534122],
[84, 182952844329494181838848],
[85, 255227018229957765044016],
[86, 355376219286719031156736],
[87, 493899311443420857952845],
[88, 685157678128482627354624],
[89, 948763597225844233250504],
[90, 1311456320500974276980736],
[91, 1809633323386495729057992],
[92, 2492760414984152205361152],
[93, 3427959082742197097793024],
[94, 4706168520874397834575872],
[95, 6450411048962389429976770],
[96, 8826863296640622526464000],
[97, 12059665023346371597383976],
[98, 16450700321708764061517824],
[99, 22405985191605778318221966],
[100, 30470821082605141257437184]]

def mcKayThompson(n):return mt[n+1][1]

def sumOfRoots(f,prime,power):
    answer=0
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    #U.<x> = T[] -- this line gave an error
    poly = U(f)
    expr=str(poly)
    var('x')
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+a
    return answer

def productOfNonzeroRoots(f,prime,power):
    answer=1
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    #U.<x> = T[] -- this line gave an error
    poly = U(f)
    expr=str(poly)
    var('x')
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer*a
    return answer


def coercePolynomial(f,prime,power):
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    return poly

def constantTerm(f,prime,power):
    cp=coercePolynomial(f,prime,power)
    expr=str(cp)
    var('x')
    expr3=sage_eval(expr,locals={'x':0})
    return expr3


def roots(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    expr3=sage_eval(expr,locals={'x':0})
    if expr3==0:answer=answer+[0]
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+[a]
    return answer

def numericalTermOverFieldNoPrint(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    xproduct=1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        xproduct=xproduct*monomial^exponent
    return g/xproduct

def polynomialDegree(poly):
    return len(poly.list())-1

def leadingCoefficient(poly):
    dp=D(poly)
    deg=polynomialDegree(dp)
    return dp.list()[deg]

def testRootLogTimesPrimeCn(f,prime,power):
    tf=True
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            j=i*prime
            b=t^j
            expr4=sage_eval(expr,locals={'x':b})
            if expr4!=0:
                tf=False
                print (i,j)
                
    return tf

def basicRoots(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        if mod(i,prime)!=0:
            a=t^i
            expr3=sage_eval(expr,locals={'x':a})
            if expr3 == 0:answer=answer+[i]
    return answer

def basicRootsDigits(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        if mod(i,prime)!=0:
            a=t^i
            expr3=sage_eval(expr,locals={'x':a})
            if expr3 == 0:answer=answer+[padListOnRightToLengthK(i.digits(prime),power)]
    return answer

def basicLogsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if mod(i,prime)!=0:
                        if b^i==xm:data=data+[[i,exponent]]
    return data.sort()

def totalMultiplicityOverField(f,prime,power):
    tm=0
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:tm=tm+exponent
    return tm

def testRootLogTimesPrimeCn(f,prime,power):
    tf=True
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            j=i*prime
            b=t^j
            expr4=sage_eval(expr,locals={'x':b})
            if expr4!=0:
                tf=False
                print (i,j)
                
    return tf


def squeezePolynomial(f,prime,power):
    dg=f.degree()
    answer=0
    for k in [0..dg]:
        j=mod(k,prime^power-1)
        cf=polynomialCoefficient(k,f)
        cf=ZZ(mod(cf,prime))
        term=ZR(cf*x^j)
        answer=answer+term
    return answer

def completeSqueezePolynomial(f,prime,power):
    sp=squeezePolynomial(f,prime,power)
    dg=sp.degree()
    answer=0
    for k in [0..dg]:
        cf=polynomialCoefficient(k,sp)
        cf=ZZ(mod(cf,prime))
        term=cf*x^k
        answer=answer+term
    print answer
    return answer

def frobenius(prime,x):return x^prime

def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf

def testIncreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]<list[k-1]:tf=False
    return tf

In [ ]:
## printed output has been snipped
# generated in `j from scratch.ipynb';
# which employs a *different* code dictionary,

import pickle


data=[]
for m in [3..500]:
    if 10 in divisors(m):
        print(m)
    data=data+[[m,j(120,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no6.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

import pickle
rfile = open('/Users/barrybrent/8mar21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/10mar21no8.txt','w')
enoughfile=[]
for qpower in [0..120]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=x*s[k][1] 
# times x because, to use polynomialCoefficient, I need a polynomial; not a Laurent series
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower-1,rl]] 
    finish = time.time()
    print([qpower,finish-start])
# -1 because I want to record the coefficients of the qpowers of the original Laurent series
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()


# print-output snipped

sum

In [3]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)

121

In [4]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
badns=[]
for k in [1..120]:
    n=s[k][0]
    poly=D(s[k][1])
    polynum=ZR(polynomialnumerator(D(poly)))
    polyden=ZR(polynomialdenominator(D(poly)))
    sd=leastSplittingDegree(polynum,prime,bound)
    print "---------------------------------------------------------------------"
    print "n:",n," splitting degree: ",sd
    print "denominator:";print factor(polyden)

---------------------------------------------------------------------
n: 0  splitting degree:  2
denominator:
1
---------------------------------------------------------------------
n: 1  splitting degree:  4
denominator:
1
---------------------------------------------------------------------
n: 2  splitting degree:  4
denominator:
3^3
---------------------------------------------------------------------
n: 3  splitting degree:  6
denominator:
1
---------------------------------------------------------------------
n: 4  splitting degree:  1
denominator:
3^3 * 5^3
---------------------------------------------------------------------
n: 5  splitting degree:  1
denominator:
3^6
---------------------------------------------------------------------
n: 6  splitting degree:  4
denominator:
3^2 * 5^3 * 7^3
---------------------------------------------------------------------
n: 7  splitting degree:  4
denominator:
3^6 * 5^3
---------------------------------------------------------------------


---------------------------------------------------------------------
n: 53  splitting degree:  6
denominator:
3^54 * 5^32 * 7^22 * 11^12 * 13^12 * 17^9 * 19^6 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3 * 47^3
---------------------------------------------------------------------
n: 54  splitting degree:  1
denominator:
3^46 * 5^35 * 7^22 * 11^15 * 13^12 * 17^9 * 19^6 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3 * 47^3 * 53^3
---------------------------------------------------------------------
n: 55  splitting degree:  1
denominator:
3^53 * 5^26 * 7^25 * 11^8 * 13^12 * 17^9 * 19^6 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3 * 47^3 * 53^3
---------------------------------------------------------------------
n: 56  splitting degree:  4
denominator:
3^57 * 5^35 * 7^17 * 11^15 * 13^12 * 17^9 * 19^9 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3 * 47^3 * 53^3
---------------------------------------------------------------------
n: 57  splitting degree:  4
denominator:
3^54 * 5^35 * 7^25 * 11^15 * 13^1

---------------------------------------------------------------------
n: 92  splitting degree:  4
denominator:
3^93 * 5^57 * 7^40 * 11^24 * 13^21 * 17^15 * 19^12 * 23^6 * 29^9 * 31^9 * 37^6 * 41^6 * 43^6 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3
---------------------------------------------------------------------
n: 93  splitting degree:  6
denominator:
3^90 * 5^57 * 7^40 * 11^24 * 13^21 * 17^15 * 19^12 * 23^12 * 29^9 * 31^4 * 37^6 * 41^6 * 43^6 * 47^6 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3
---------------------------------------------------------------------
n: 94  splitting degree:  1
denominator:
3^92 * 5^60 * 7^40 * 11^24 * 13^21 * 17^15 * 19^15 * 23^12 * 29^9 * 31^9 * 37^6 * 41^6 * 43^6 * 47^2 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3
---------------------------------------------------------------------
n: 95  splitting degree:  1
denominator:
3^95 * 5^45 * 7^40 * 11^24 * 13^21 * 17^15 * 19^8 * 23^12 * 29^9 

In [5]:
def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf
        
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
badns=[]
for k in [1..20]:
    n=s[k][0]
    poly=D(s[k][1])
    polynum=ZR(polynomialnumerator(D(poly)))
    polyden=ZR(polynomialdenominator(D(poly)))
    orders=[]
    for p in [2..(n+1)]:
        if is_prime(p)==True:
            op=ord(p,polyden)
            orders=orders+[op]
    tf=testDecreasing(orders)
    if tf==False:
        badns=badns+[n]
        print factor(polyden)
print badns

3^3
3^3 * 5^3
3^6
3^2 * 5^3 * 7^3
3^6 * 5^3
3^8 * 5^3 * 7^3
3^4 * 5^6 * 7^3
3^8 * 5^2 * 7^3 * 11^3
3^11 * 5^6 * 7^3
3^9 * 5^6 * 7^3 * 11^3 * 13^3
3^12 * 5^6 * 7^6 * 11^3
3^15 * 5^9 * 7^2 * 11^3 * 13^3
3^11 * 5^4 * 7^6 * 11^3 * 13^3
3^15 * 5^9 * 7^6 * 11^3 * 13^3 * 17^3
3^16 * 5^9 * 7^6 * 11^3 * 13^3
3^12 * 5^9 * 7^6 * 11^3 * 13^3 * 17^3 * 19^3
3^18 * 5^12 * 7^6 * 11^3 * 13^3 * 17^3
[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [7]:
def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf
        
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
badns=[]
for k in [1..120]:
    n=s[k][0]
    print n
    poly=D(s[k][1])
    polynum=ZR(polynomialnumerator(D(poly)))
    polyden=ZR(polynomialdenominator(D(poly)))
    orders=[]
    for p in [2..(n+1)]:
        if is_prime(p)==True:
            op=ord(p,polyden)
            orders=orders+[op]
    tf=testDecreasing(orders)
    if tf==False:
        badns=badns+[n]
        print factor(polyden)
print badns

0
1
2
3^3
3
4
3^3 * 5^3
5
3^6
6
3^2 * 5^3 * 7^3
7
3^6 * 5^3
8
3^8 * 5^3 * 7^3
9
3^4 * 5^6 * 7^3
10
3^8 * 5^2 * 7^3 * 11^3
11
3^11 * 5^6 * 7^3
12
3^9 * 5^6 * 7^3 * 11^3 * 13^3
13
3^12 * 5^6 * 7^6 * 11^3
14
3^15 * 5^9 * 7^2 * 11^3 * 13^3
15
3^11 * 5^4 * 7^6 * 11^3 * 13^3
16
3^15 * 5^9 * 7^6 * 11^3 * 13^3 * 17^3
17
3^16 * 5^9 * 7^6 * 11^3 * 13^3
18
3^12 * 5^9 * 7^6 * 11^3 * 13^3 * 17^3 * 19^3
19
3^18 * 5^12 * 7^6 * 11^3 * 13^3 * 17^3
20
3^21 * 5^6 * 7^9 * 11^3 * 13^3 * 17^3 * 19^3
21
3^18 * 5^12 * 7^4 * 11^6 * 13^3 * 17^3 * 19^3
22
3^20 * 5^12 * 7^9 * 11^2 * 13^3 * 17^3 * 19^3 * 23^3
23
3^23 * 5^12 * 7^9 * 11^6 * 13^3 * 17^3 * 19^3
24
3^20 * 5^16 * 7^9 * 11^6 * 13^3 * 17^3 * 19^3 * 23^3
25
3^24 * 5^8 * 7^9 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3
26
3^27 * 5^16 * 7^9 * 11^6 * 13^2 * 17^3 * 19^3 * 23^3
27
3^20 * 5^16 * 7^12 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3
28
3^27 * 5^16 * 7^5 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3 * 29^3
29
3^30 * 5^19 * 7^12 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3
30
3^25 * 5^13 * 7^

3^95 * 5^45 * 7^40 * 11^24 * 13^21 * 17^15 * 19^8 * 23^12 * 29^9 * 31^9 * 37^6 * 41^6 * 43^6 * 47^6 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3
96
3^91 * 5^60 * 7^40 * 11^24 * 13^21 * 17^15 * 19^15 * 23^12 * 29^9 * 31^9 * 37^6 * 41^6 * 43^6 * 47^6 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3
97
3^96 * 5^60 * 7^44 * 11^24 * 13^21 * 17^15 * 19^15 * 23^12 * 29^9 * 31^9 * 37^6 * 41^6 * 43^6 * 47^6 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3
98
3^99 * 5^60 * 7^27 * 11^27 * 13^21 * 17^15 * 19^15 * 23^12 * 29^9 * 31^9 * 37^6 * 41^6 * 43^6 * 47^6 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3
99
3^93 * 5^64 * 7^44 * 11^16 * 13^21 * 17^15 * 19^15 * 23^12 * 29^9 * 31^9 * 37^6 * 41^6 * 43^6 * 47^6 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3
100
3^99 * 5^44 * 7^44 * 11^27 * 13^21 * 17^15 * 19^15 * 23^12 * 29^9 * 31^9 * 37^6 * 41^6 * 43^6 * 47^6 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 *

In [9]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
o2=[]
for k in [1..120]:
    n=s[k][0]
    if is_prime(n)==True:
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        o2=o2+[ord(2,polyden)]
print o2

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
o3=[]
for k in [1..120]:
    n=s[k][0]
    if is_prime(n)==True:
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        o3=o3+[ord(3,polyden)]
print o3

[3, 0, 6, 6, 11, 12, 16, 18, 23, 30, 30, 36, 42, 42, 48, 54, 60, 60, 66, 72, 72, 78, 83, 90, 96, 101, 102, 108, 108, 114]


In [14]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
o3=[]
for k in [1..120]:
    n=s[k][0]
    if is_prime(n)==True:
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        o3=o3+[ord(3,polyden)]
print o3



[3, 0, 6, 6, 11, 12, 16, 18, 23, 30, 30, 36, 42, 42, 48, 54, 60, 60, 66, 72, 72, 78, 83, 90, 96, 101, 102, 108, 108, 114]


In [15]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
o5=[]
for k in [1..120]:
    oldpolyden=1
    n=s[k][0]
    if is_prime(n)==True:
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        quotient=ZR(polyden/oldpolyden)
        oldpolyden=polyden
        print "==============================================================="
        print "n: ",n
        print factor(quotient)

n:  2
3^3
n:  3
1
n:  5
3^6
n:  7
3^6 * 5^3
n:  11
3^11 * 5^6 * 7^3
n:  13
3^12 * 5^6 * 7^6 * 11^3
n:  17
3^16 * 5^9 * 7^6 * 11^3 * 13^3
n:  19
3^18 * 5^12 * 7^6 * 11^3 * 13^3 * 17^3
n:  23
3^23 * 5^12 * 7^9 * 11^6 * 13^3 * 17^3 * 19^3
n:  29
3^30 * 5^19 * 7^12 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3
n:  31
3^30 * 5^19 * 7^12 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3 * 29^3
n:  37
3^36 * 5^22 * 7^15 * 11^9 * 13^6 * 17^6 * 19^6 * 23^3 * 29^3 * 31^3
n:  41
3^42 * 5^25 * 7^18 * 11^9 * 13^9 * 17^6 * 19^6 * 23^3 * 29^3 * 31^3 * 37^3
n:  43
3^42 * 5^25 * 7^18 * 11^12 * 13^9 * 17^6 * 19^6 * 23^3 * 29^3 * 31^3 * 37^3 * 41^3
n:  47
3^48 * 5^28 * 7^18 * 11^12 * 13^9 * 17^6 * 19^6 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3
n:  53
3^54 * 5^32 * 7^22 * 11^12 * 13^12 * 17^9 * 19^6 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3 * 47^3
n:  59
3^60 * 5^38 * 7^25 * 11^15 * 13^12 * 17^9 * 19^9 * 23^6 * 29^6 * 31^3 * 37^3 * 41^3 * 43^3 * 47^3 * 53^3
n:  61
3^60 * 5^38 * 7^25 * 11^15 * 13^12 * 17^9 * 19^9 * 23^6 * 29^6 * 31

In [22]:
def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf
        
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
badns=[]
for k in [1..120]:
    n=s[k][0]
    if (is_prime(n)==True)&(n>2):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        orders=[]
        for p in [3..(n+1)]:
            if is_prime(p)==True:
                op=ord(p,polyden)
                orders=orders+[op]
        tf=testDecreasing(orders)
        if tf==False:
            badns=badns+[n]
            print factor(polyden)
print badns

[]


In [29]:
def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf
        
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
badns=[]
for k in [1..120]:
    n=s[k][0]
    if (is_prime(n)==True)&(1==1):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        orders=[]
        for p in [3..(n+1)]:
            if is_prime(p)==True:
                op=ord(p,polyden)
                orders=orders+[op]
        tf=testDecreasing(orders)
        if tf==False:
            badns=badns+[n]
            print factor(polyden)
print badns

[]


In [30]:
def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf
        
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
badns=[]
for k in [1..120]:
    n=s[k][0]
    if (is_prime(n)==True)&(1==1):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        orders=[]
        for p in [2..(n+1)]:
            if is_prime(p)==True:
                op=ord(p,polyden)
                orders=orders+[op]
        tf=testDecreasing(orders)
        if tf==False:
            badns=badns+[n]
            print factor(polyden)
print badns

3^3
3^6
3^6 * 5^3
3^11 * 5^6 * 7^3
3^12 * 5^6 * 7^6 * 11^3
3^16 * 5^9 * 7^6 * 11^3 * 13^3
3^18 * 5^12 * 7^6 * 11^3 * 13^3 * 17^3
3^23 * 5^12 * 7^9 * 11^6 * 13^3 * 17^3 * 19^3
3^30 * 5^19 * 7^12 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3
3^30 * 5^19 * 7^12 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3 * 29^3
3^36 * 5^22 * 7^15 * 11^9 * 13^6 * 17^6 * 19^6 * 23^3 * 29^3 * 31^3
3^42 * 5^25 * 7^18 * 11^9 * 13^9 * 17^6 * 19^6 * 23^3 * 29^3 * 31^3 * 37^3
3^42 * 5^25 * 7^18 * 11^12 * 13^9 * 17^6 * 19^6 * 23^3 * 29^3 * 31^3 * 37^3 * 41^3
3^48 * 5^28 * 7^18 * 11^12 * 13^9 * 17^6 * 19^6 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3
3^54 * 5^32 * 7^22 * 11^12 * 13^12 * 17^9 * 19^6 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3 * 47^3
3^60 * 5^38 * 7^25 * 11^15 * 13^12 * 17^9 * 19^9 * 23^6 * 29^6 * 31^3 * 37^3 * 41^3 * 43^3 * 47^3 * 53^3
3^60 * 5^38 * 7^25 * 11^15 * 13^12 * 17^9 * 19^9 * 23^6 * 29^6 * 31^6 * 37^3 * 41^3 * 43^3 * 47^3 * 53^3 * 59^3
3^66 * 5^41 * 7^28 * 11^18 * 13^15 * 17^12 * 19^9 * 23^6 * 29^6 * 31^6 * 37^3 *

In [34]:
def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf
        
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
one=[]
three=[]
badns=[]
for k in [1..120]:
    n=s[k][0]
   
    poly=D(s[k][1])
    polynum=ZR(polynomialnumerator(D(poly)))
    polyden=ZR(polynomialdenominator(D(poly)))
    md4=mod(polyden,4)
    if md4==1:one=one+[n]
    if md4==3:three=three+[n]
print "one:";print one;print "three:";print three

one:
[0, 1, 3, 5, 7, 10, 11, 14, 15, 16, 18, 20, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 47, 48, 52, 54, 57, 59, 61, 62, 63, 64, 65, 66, 67, 70, 72, 73, 76, 77, 78, 80, 81, 83, 88, 89, 90, 92, 96, 98, 99, 103, 104, 105, 107, 108, 109, 112, 113, 117, 119]
three:
[2, 4, 6, 8, 9, 12, 13, 17, 19, 21, 22, 23, 24, 25, 32, 41, 45, 46, 49, 50, 51, 53, 55, 56, 58, 60, 68, 69, 71, 74, 75, 79, 82, 84, 85, 86, 87, 91, 93, 94, 95, 97, 100, 101, 102, 106, 110, 111, 114, 115, 116, 118]


In [36]:
def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf
        
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
one=[]
zero=[]
badns=[]
for k in [1..120]:
    n=s[k][0]
   
    poly=D(s[k][1])
    polynum=ZR(polynomialnumerator(D(poly)))
    polyden=ZR(polynomialdenominator(D(poly)))
    md2=mod(polyden,2)
    if md2==1:one=one+[n]
    if md2==0:zero=zero+[n]
print "one:";print one;print "zero:";print zero

one:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119]
zero:
[]


In [23]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
def mod3(n):return mod(n,3)
for k in [1..120]:
    n=s[k][0]
    if (is_prime(n)==True)&(n>3):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        orders=[]
        for p in [3..(n+1)]:
            if is_prime(p)==True:
                op=ord(p,polyden)
                orders=orders+[op]
        print orders
        print map(mod3,orders)
        print "--------------------------------------------------------------------------------------------"
    

[6, 0]
[0, 0]
--------------------------------------------------------------------------------------------
[6, 3, 0]
[0, 0, 0]
--------------------------------------------------------------------------------------------
[11, 6, 3, 0]
[2, 0, 0, 0]
--------------------------------------------------------------------------------------------
[12, 6, 6, 3, 0]
[0, 0, 0, 0, 0]
--------------------------------------------------------------------------------------------
[16, 9, 6, 3, 3, 0]
[1, 0, 0, 0, 0, 0]
--------------------------------------------------------------------------------------------
[18, 12, 6, 3, 3, 3, 0]
[0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------------------------------------------------
[23, 12, 9, 6, 3, 3, 3, 0]
[2, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------------------------------------------------
[30, 19, 12, 6, 6, 3, 3, 3, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
-----------------------------------------------------------

In [24]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
o5=[]
for k in [1..120]:
    oldpolyden=1
    n=s[k][0]
    poly=D(s[k][1])
    polynum=ZR(polynomialnumerator(D(poly)))
    polyden=ZR(polynomialdenominator(D(poly)))
    quotient=ZR(polyden/oldpolyden)
    oldpolyden=polyden
    print "==============================================================="
    print "n: ",n
    print factor(quotient)



n:  0
1
n:  1
1
n:  2
3^3
n:  3
1
n:  4
3^3 * 5^3
n:  5
3^6
n:  6
3^2 * 5^3 * 7^3
n:  7
3^6 * 5^3
n:  8
3^8 * 5^3 * 7^3
n:  9
3^4 * 5^6 * 7^3
n:  10
3^8 * 5^2 * 7^3 * 11^3
n:  11
3^11 * 5^6 * 7^3
n:  12
3^9 * 5^6 * 7^3 * 11^3 * 13^3
n:  13
3^12 * 5^6 * 7^6 * 11^3
n:  14
3^15 * 5^9 * 7^2 * 11^3 * 13^3
n:  15
3^11 * 5^4 * 7^6 * 11^3 * 13^3
n:  16
3^15 * 5^9 * 7^6 * 11^3 * 13^3 * 17^3
n:  17
3^16 * 5^9 * 7^6 * 11^3 * 13^3
n:  18
3^12 * 5^9 * 7^6 * 11^3 * 13^3 * 17^3 * 19^3
n:  19
3^18 * 5^12 * 7^6 * 11^3 * 13^3 * 17^3
n:  20
3^21 * 5^6 * 7^9 * 11^3 * 13^3 * 17^3 * 19^3
n:  21
3^18 * 5^12 * 7^4 * 11^6 * 13^3 * 17^3 * 19^3
n:  22
3^20 * 5^12 * 7^9 * 11^2 * 13^3 * 17^3 * 19^3 * 23^3
n:  23
3^23 * 5^12 * 7^9 * 11^6 * 13^3 * 17^3 * 19^3
n:  24
3^20 * 5^16 * 7^9 * 11^6 * 13^3 * 17^3 * 19^3 * 23^3
n:  25
3^24 * 5^8 * 7^9 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3
n:  26
3^27 * 5^16 * 7^9 * 11^6 * 13^2 * 17^3 * 19^3 * 23^3
n:  27
3^20 * 5^16 * 7^12 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3
n:  28
3^27 * 5^16 * 

n:  78
3^74 * 5^48 * 7^34 * 11^21 * 13^10 * 17^12 * 19^12 * 23^9 * 29^6 * 31^6 * 37^6 * 41^3 * 43^3 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3
n:  79
3^78 * 5^51 * 7^34 * 11^21 * 13^18 * 17^12 * 19^12 * 23^9 * 29^6 * 31^6 * 37^6 * 41^3 * 43^3 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3
n:  80
3^80 * 5^39 * 7^34 * 11^21 * 13^18 * 17^12 * 19^12 * 23^9 * 29^6 * 31^6 * 37^6 * 41^3 * 43^3 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3
n:  81
3^72 * 5^51 * 7^34 * 11^21 * 13^18 * 17^12 * 19^12 * 23^9 * 29^6 * 31^6 * 37^6 * 41^6 * 43^3 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3
n:  82
3^80 * 5^51 * 7^34 * 11^21 * 13^18 * 17^12 * 19^12 * 23^9 * 29^6 * 31^6 * 37^6 * 41^2 * 43^3 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3
n:  83
3^83 * 5^51 * 7^37 * 11^21 * 13^18 * 17^12 * 19^12 * 23^9 * 29^6 * 31^6 * 37^6 * 41^6 * 43^3 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3
n:  84
3^81 * 5^54 * 7^25 * 11^21 * 13^18 * 17^15 * 19^12 * 23^9

n:  117
3^112 * 5^73 * 7^50 * 11^30 * 13^16 * 17^18 * 19^18 * 23^15 * 29^12 * 31^9 * 37^9 * 41^6 * 43^6 * 47^6 * 53^6 * 59^6 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3
n:  118
3^116 * 5^73 * 7^53 * 11^30 * 13^27 * 17^21 * 19^18 * 23^15 * 29^12 * 31^9 * 37^9 * 41^6 * 43^6 * 47^6 * 53^6 * 59^2 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3
n:  119
3^120 * 5^76 * 7^38 * 11^30 * 13^27 * 17^12 * 19^18 * 23^15 * 29^12 * 31^9 * 37^9 * 41^6 * 43^6 * 47^6 * 53^6 * 59^6 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3


In [31]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [2..120]:
    n=s[k][0]
    
    poly=D(s[k][1])
  
    polyden=ZR(polynomialdenominator(D(poly)))
    if polyden>1:
        lep=lastElement(prime_divisors(polyden))
        np=next_prime(lep)
        mnp=ZR(mod(polyden,np))
        if is_prime(n)==True:
            print (n,mnp)
            data=data+[mnp]
print data

(2, 2)
(5, 4)
(7, 6)
(11, 8)
(13, 5)
(17, 1)
(19, 8)
(23, 11)
(29, 1)
(31, 13)
(37, 18)
(41, 22)
(43, 20)
(47, 26)
(53, 42)
(59, 6)
(61, 24)
(67, 21)
(71, 52)
(73, 54)
(79, 53)
(83, 37)
(89, 88)
(97, 91)
(101, 85)
(103, 61)
(107, 96)
(109, 8)
(113, 46)
[2, 4, 6, 8, 5, 1, 8, 11, 1, 13, 18, 22, 20, 26, 42, 6, 24, 21, 52, 54, 53, 37, 88, 91, 85, 61, 96, 8, 46]


In [32]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [2..10]:
    n=s[k][0]
    
    poly=D(s[k][1])
  
    polyden=ZR(polynomialdenominator(D(poly)))
    if polyden>1:
        pd=prime_divisors(polyden)
        lp=lastElement(pd)
        np=next_prime(lp)
        mp=ZR(mod(polyden,np))
        print [n,mp,polyden,polyden-mp]
    

[2, 2, 27, 25]
[4, 1, 3375, 3374]
[5, 4, 729, 725]
[6, 6, 385875, 385869]
[7, 6, 91125, 91119]
[8, 7, 281302875, 281302868]
[9, 7, 434109375, 434109368]
